<a href="https://colab.research.google.com/github/SoumyaAIDev/AI_projects/blob/main/Medical_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


In [2]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
Medicine_Meta_Llama_LoRA = "AhmedBou/Arabic-Medicine-Meta-Llama-3.2-3B-LoRA"
Law_Meta_Llama_LoRA = "AhmedBou/Arabic-Law-Meta-Llama-3.2-3B-LoRA"


In [4]:
medicine_model, medicine_tokenizer = FastLanguageModel.from_pretrained(
    model_name= Medicine_Meta_Llama_LoRA,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

# Enable faster inference
FastLanguageModel.for_inference(medicine_model)

==((====))==  Unsloth 2025.2.4: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Unsloth 2025.2.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [5]:
law_model, law_tokenizer = FastLanguageModel.from_pretrained(
    model_name= Law_Meta_Llama_LoRA,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

# Enable faster inference
FastLanguageModel.for_inference(law_model)

==((====))==  Unsloth 2025.2.4: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following question in Arabic:

### Input:
{}

### Response:
{}"""

In [7]:
def parse_response(output):
    # Ensure the output is a string
    if isinstance(output, list):
        output = output[0]

    # Define the start and end markers
    start_marker = "### Response:\n"
    end_marker = "<|eot_id|>"

    # Extract the response between the markers
    if start_marker in output and end_marker in output:
        start_index = output.index(start_marker) + len(start_marker)
        end_index = output.index(end_marker)
        response = output[start_index:end_index].strip()
        return response
    return None

In [8]:
def get_Medicine_answers(question, alpaca_prompt=alpaca_prompt, model=medicine_model, tokenizer=medicine_tokenizer):

    # Prepare the input using the tokenizer
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                question,  # Question is passed here
                "",        # Output - leave this blank for generation
            )
        ], return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 128, temperature=0.1, use_cache = True)
    tokenizer.batch_decode(outputs)
    return parse_response(tokenizer.batch_decode(outputs))

In [9]:
def get_Law_answers(question, alpaca_prompt=alpaca_prompt, model=law_model, tokenizer=law_tokenizer):

    # Prepare the input using the tokenizer
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                question,  # Question is passed here
                "",        # Output - leave this blank for generation
            )
        ], return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 128, temperature=0.1, use_cache = True)
    tokenizer.batch_decode(outputs)
    return parse_response(tokenizer.batch_decode(outputs))

In [10]:
res = get_Law_answers("ما هو الفرق بين القانون المدني والقانون الجنائي؟")
print(res)

القانون المدني يركز على الحقوق والالتزامات بين الأفراد، بينما يركز القانون الجنائي على الجريمة والتعويض.


In [12]:
medicine_tool={
    "type": "function",
    "function": {
        "name": "get_Medicine_answers",
        "description": "Provides medical information and answers to health-related queries.",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The medical question or topic needing an answer."
                }
            },
            "required": ["question"]
        }
    }
}

law_tool={
    "type": "function",
    "function": {
        "name": "get_Law_answers",
        "description": "Provides legal information and answers to law-related queries.",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The legal question or topic needing an answer."
                }
            },
            "required": ["question"]
        }
    }
}

In [13]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.9 MB/s eta 0:00:00


In [18]:
import os
from groq import Groq
from google.colab import userdata
import json




In [30]:
import os

os.environ["GROQ_API_KEY"] = "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"


In [31]:
from groq import Groq

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

print("Groq API Key Set Successfully!")


Groq API Key Set Successfully!


In [ ]:
history = []

# Loop for chat
while True:
    # Get user input
    user_input = input("You: ")

    # Exit the loop if the user types 'exit'
    if user_input.lower() == "exit":
        break

    # Append the user's input to the history
    history.append({"role": "user", "content": user_input})

    # Make the chat completion request with the current history
    chat_completion = client.chat.completions.create(
        messages=history,
        model="llama-3.2-90b-vision-preview",
    )

    # Get the assistant's response
    assistant_response = chat_completion.choices[0].message.content
    print(f"Assistant: {assistant_response}")

    # Append the assistant's response to the history
    history.append({"role": "assistant", "content": assistant_response})



You: hi
Assistant: Hello. How can I help you today?
You: how complete details about medicine
Assistant: Medicine is a vast and complex field, and I'll do my best to provide you with an overview of the different aspects. Here's a comprehensive guide to get you started:

**Definition and Types of Medicine**

Medicine is the science and art of preventing, diagnosing, and treating diseases and disorders. There are several types of medicine, including:

1. **Allopathic Medicine**: Also known as conventional medicine, it involves the use of pharmaceuticals, surgery, and other medical interventions to treat diseases.
2. **Alternative Medicine**: This includes non-conventional therapies such as acupuncture, herbal medicine, and homeopathy.
3. **Holistic Medicine**: Focuses on treating the whole person, including physical, mental, and spiritual well-being.
4. **Integrative Medicine**: Combines conventional and alternative therapies to create a comprehensive treatment plan.

**Branches of Medici